In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam

In [2]:
# Enable mixed precision training
from tensorflow.keras.mixed_precision import set_global_policy
set_global_policy('mixed_float16')


The dtype policy mixed_float16 may run slowly because this machine does not have a GPU. Only Nvidia GPUs with compute capability of at least 7.0 run quickly with mixed_float16.
If you will use compatible GPU(s) not attached to this host, e.g. by running a multi-worker model, you can ignore this warning. This message will only be logged once


In [4]:
# Define paths to your dataset
train_data_dir = '/Users/sujaykaushik/Documents/Msc DataScience/Project Dissertation/All Files (Code & Dataset)/Sign_Lang_Custom_Dataset _Final copy'
validation_data_dir = '/Users/sujaykaushik/Documents/Msc DataScience/Project Dissertation/All Files (Code & Dataset)/Sign_Lang_Custom_Dataset _Final copy'

In [36]:
# Image dimensions
img_height, img_width = 128,128
batch_size = 32  # Increased batch size for faster training

In [37]:
# Data augmentation and normalization
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

validation_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)

validation_generator = validation_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)

Found 125627 images belonging to 36 classes.
Found 125627 images belonging to 36 classes.


In [38]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [39]:
# Load the MobileNetV2 model without the top layers
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))
# Add custom layers on top of the base model
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(train_generator.num_classes, activation='softmax')(x)

9406464/9406464 [==============================] - 1s 0us/step


In [40]:
# Define the complete model
model = Model(inputs=base_model.input, outputs=predictions)


In [41]:
# Freeze the base layers
for layer in base_model.layers:
    layer.trainable = False

In [42]:
# Compile the model with mixed precision policy
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [43]:
# Specify the path to save the entire model
model_save_path = 'All Files (Code & Dataset)/MyModel/Save1_Mobilenet.h5'

In [44]:
# Create a ModelCheckpoint callback to save the model during training
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    model_save_path, verbose=1, save_weights_only=False
)

In [45]:
# Create an EarlyStopping callback to stop training early if the validation loss doesn't improve
es_callback = tf.keras.callbacks.EarlyStopping(patience=80, verbose=1)

In [46]:
# Function to print a message every 5 minutes
import time
def print_keep_alive():
    while True:
        print("Colab session is active.")
        time.sleep(300)  # Wait for 5 minutes

In [47]:
# Start the keep-alive loop in the background
import threading
keep_alive_thread = threading.Thread(target=print_keep_alive)
keep_alive_thread.daemon = True
keep_alive_thread.start()


Colab session is active.


In [48]:
# Training the model
history = model.fit(train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size,
    epochs=100,
    callbacks=[cp_callback, es_callback]
)

Epoch 1/100
3925/3925 [==============================] - ETA: 0s - loss: 0.1039 - accuracy: 0.9689Colab session is active.
Colab session is active.
Colab session is active.
Colab session is active.
Colab session is active.
Colab session is active.

Epoch 1: saving model to All Files (Code & Dataset)/MyModel/Save1_Mobilenet.h5
3925/3925 [==============================] - 1296s 330ms/step - loss: 0.1039 - accuracy: 0.9689 - val_loss: 0.1821 - val_accuracy: 0.9408
Epoch 2/100
3925/3925 [==============================] - ETA: 0s - loss: 0.0444 - accuracy: 0.9867Colab session is active.
Colab session is active.
Colab session is active.
Colab session is active.
Colab session is active.
Colab session is active.

Epoch 2: saving model to All Files (Code & Dataset)/MyModel/Save1_Mobilenet.h5
3925/3925 [==============================] - 1268s 323ms/step - loss: 0.0444 - accuracy: 0.9867 - val_loss: 0.1563 - val_accuracy: 0.9604
Epoch 3/100
3925/3925 [==============================] - ETA: 0s - l

KeyboardInterrupt: 